In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms

from modules.UNet import *
from modules.Discriminator import *
from modules.DataSet import *
from modules.Losses import *

import os,sys

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
from PIL import Image

import numpy as np

## Training

In [2]:
def train_adversarial(net,discriminator,device,val_per=0.1,epochs=10,batch_size=10,resize_to=None,alpha=1):
    if resize_to is not None:
        transform_image = transforms.Compose([
        transforms.Resize(resize_to),
        transforms.ToTensor(),
        transforms.Normalize(0.5,0.5)
        ])
        transform_label = transforms.Compose([
        transforms.Resize(resize_to),
        transforms.ToTensor()
        ])
    else:
        transform_image = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(0.5,0.5)
        ])
        transform_label = transforms.Compose([
        transforms.ToTensor()
        ])
            
    
    dataSet = UltraSoundDataSet(root_dir,(transform_image,transform_label))
    nTrain = int(len(dataSet)*(1-val_per))
    nValid = int(len(dataSet)-nTrain)
    
    trainSet,validSet = random_split(dataSet,[nTrain,nValid])
    
    train_loader = DataLoader(trainSet,batch_size=batch_size,shuffle=True,num_workers=4)
    valid_loader = DataLoader(validSet,batch_size=batch_size,shuffle=True,num_workers=4)
    
    optimizer_unet = torch.optim.Adam(net.parameters())
    optimizer_disc = torch.optim.Adam(discriminator.parameters())
    scheduler_unet = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_unet,mode='min',patience=10) #mae: dice-index
    scheduler_disc = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_disc,mode='min',patience=10)
    BCELoss = nn.BCELoss()
    
    running_loss_seg = 0
    running_loss_disc = 0
    step = 0
    np.set_printoptions(precision=2)
    
    isAdv = True;
    for epoch in range(epochs):
        net.train()
        discriminator.train()
        
        for batch in train_loader:
            imgs,labels = batch
            
            imgs = imgs.to(device=device,dtype=torch.float32)
            labels = labels.to(device=device,dtype=torch.float32)
            
            valid = torch.Tensor(imgs.size(0), 1).fill_(1.0).to(device=device,dtype=torch.float32)
            fake = torch.Tensor(imgs.size(0), 1).fill_(0.0).to(device=device,dtype=torch.float32)
            
            if(isAdv):
                # Train adversarial 
                with torch.no_grad():
                    # segmentation
                    pred = net(imgs)
                pred_valid = discriminator(imgs, labels)
                pred_fake = discriminator(imgs, pred.detach())
                print("valid: ",np.array(pred_valid.view([1,-1]).tolist()),"; fake: ",np.array(pred_fake.view([1,-1]).tolist()) )
                
                # discriminator loss
                disc_loss = BCELoss(pred_valid,valid) + BCELoss(pred_fake,fake)
                
                optimizer_disc.zero_grad()
                disc_loss.backward()
                optimizer_disc.step()
                
                running_loss_disc += disc_loss.item()
                
            else:
                # segmentation
                pred = net(imgs)
                with torch.no_grad():
                    pred_fake = discriminator(imgs, pred)
                print("fake: ",np.array(pred_fake.view([1,-1]).tolist()) )
                
                # segmentation loss
                seg_loss = BCELoss(pred,labels) + alpha*BCELoss(pred_fake,valid)
                
                optimizer_unet.zero_grad()
                seg_loss.backward()
                optimizer_unet.step()
                
                running_loss_seg += seg_loss.item()
              
            step += 1 
            if step % 10 == 9:    # print every 10 mini-batches
                if(isAdv):
                    print('[%d, %5d] discrimination loss: %.3f' %(epoch + 1, step + 1, running_loss_disc / 10))
                    running_loss_disc = 0.0
                else:
                    print('[%d, %5d] segmentation loss: %.3f' %(epoch + 1, step + 1, running_loss_seg / 10))  
                    running_loss_seg = 0.0
                    
            if step%100 == 99:
                val_loss_seg = 0
                val_loss_disc = 0
                for batch in valid_loader:
                    imgs,labels = batch
                    imgs = imgs.to(device=device,dtype=torch.float32)
                    labels = labels.to(device=device,dtype=torch.float32)
                    
                    with torch.no_grad():
                        pred_seg = net(imgs)
                        pred_valid = discriminator(imgs, labels)
                        pred_fake = discriminator(imgs, pred_seg.detach())
                    
                    val_loss_seg += (BCELoss(pred_seg,labels) + alpha*BCELoss(pred_fake,fake)).item()
                    valid = torch.Tensor(imgs.size(0), 1).fill_(1.0).to(device=device,dtype=torch.float32)
                    fake = torch.Tensor(imgs.size(0), 1).fill_(0.0).to(device=device,dtype=torch.float32)
                    val_loss_disc += (BCELoss(pred_valid,valid) + BCELoss(pred_fake,fake)).item()
                print('[%d, %5d] validation loss(seg): %.3f; validation loss(disc): %.3f' %(epoch + 1, step + 1, val_loss_seg / len(valid_loader), val_loss_disc / len(valid_loader)))
                scheduler_unet.step(val_loss_seg)
                scheduler_disc.step(val_loss_disc)
                
                if(isAdv):
                    scheduler_disc.step(val_loss_disc)
                else:
                    scheduler_unet.step(val_loss_seg)
                
                isAdv = not isAdv
                

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = UNet(init_features=64)
discriminator = Discriminator()

net = net.to(device)
discriminator = discriminator.to(device)

IMG_SIZE = [256,256]
#IMG_SIZE = None

root_dir = os.path.expanduser("~/workspace/us_robot/DataSet/SimRealDataset")
try:
    #train(net=net,device=device,resize_to=None,epochs=50,batch_size=5)
    train_adversarial(net=net,discriminator=discriminator,device=device,resize_to=IMG_SIZE,epochs=20,batch_size=3)
except KeyboardInterrupt:
    sys.exit()

valid:  [[0.49 0.49 0.49]] ; fake:  [[0.49 0.49 0.49]]
valid:  [[0.7  0.7  0.71]] ; fake:  [[0.6  0.6  0.61]]
valid:  [[0.68 0.65 0.68]] ; fake:  [[0.5  0.51 0.52]]
valid:  [[0.73 0.75 0.74]] ; fake:  [[0.43 0.45 0.43]]
valid:  [[0.85 0.83 0.84]] ; fake:  [[0.37 0.34 0.34]]
valid:  [[0.78 0.89 0.86]] ; fake:  [[0.2  0.19 0.18]]
valid:  [[0.96 0.95 0.96]] ; fake:  [[0.1  0.15 0.1 ]]
valid:  [[0.98 0.96 0.99]] ; fake:  [[0.02 0.02 0.03]]
valid:  [[0.9  0.99 0.99]] ; fake:  [[0.01 0.01 0.01]]
[1,    10] discrimination loss: 0.592
valid:  [[1. 1. 1.]] ; fake:  [[0.01 0.01 0.01]]
valid:  [[1. 1. 1.]] ; fake:  [[0.01 0.   0.01]]
valid:  [[1. 1. 1.]] ; fake:  [[0. 0. 0.]]
valid:  [[1. 1. 1.]] ; fake:  [[0. 0. 0.]]
valid:  [[1. 1. 1.]] ; fake:  [[0. 0. 0.]]
valid:  [[1. 1. 1.]] ; fake:  [[2.51e-04 7.73e-05 8.54e-05]]
valid:  [[1. 1. 1.]] ; fake:  [[2.56e-05 6.65e-05 2.99e-05]]
valid:  [[1. 1. 1.]] ; fake:  [[1.26e-05 2.66e-05 5.95e-05]]
valid:  [[1. 1. 1.]] ; fake:  [[8.75e-06 2.21e-06 7.44e-0

fake:  [[0.24 0.12 0.12]]
fake:  [[0.2  0.32 0.74]]
fake:  [[0.05 0.05 0.05]]
fake:  [[0.7  0.88 0.47]]
fake:  [[0.68 0.28 0.19]]
fake:  [[0.23 0.16 0.32]]
fake:  [[0.08 0.09 0.08]]
[1,   160] segmentation loss: 1.986
fake:  [[0.58 0.27 0.35]]
fake:  [[0.17 0.14 0.63]]
fake:  [[0.15 0.1  0.09]]
fake:  [[0.2  0.21 0.18]]
fake:  [[0.82 0.94 0.96]]
fake:  [[0.87 0.83 0.57]]
fake:  [[0.61 0.33 0.8 ]]
fake:  [[0.18 0.42 0.34]]
fake:  [[0.13 0.76 0.29]]
fake:  [[0.3  0.55 0.18]]
[1,   170] segmentation loss: 1.202
fake:  [[0.25 0.57 0.87]]
fake:  [[0.27 0.25 0.4 ]]
fake:  [[0.26 0.88 0.49]]
fake:  [[0.71 0.55 0.63]]
fake:  [[0.62 0.61 0.61]]
fake:  [[0.95 0.95 0.74]]
fake:  [[0.56 0.97 0.56]]
fake:  [[0.88 0.91 0.99]]
fake:  [[0.81 0.95 0.97]]
fake:  [[0.93 0.97 0.71]]
[1,   180] segmentation loss: 0.580
fake:  [[0.93 0.4  0.41]]
fake:  [[0.68 0.6  0.54]]
fake:  [[0.69 0.76 0.58]]
fake:  [[0.08 0.03 0.91]]
fake:  [[0.44 0.71 0.01]]
fake:  [[0.15 0.58 0.1 ]]
fake:  [[0.8  0.78 0.55]]
fake:  [

fake:  [[1.33e-05 8.14e-03 1.55e-06]]
fake:  [[2.82e-02 9.46e-05 1.15e-01]]
fake:  [[9.51e-06 1.85e-01 9.30e-01]]
fake:  [[8.44e-01 9.81e-01 1.74e-08]]
fake:  [[1.86e-09 2.30e-02 3.40e-08]]
[1,   310] segmentation loss: 13.747
fake:  [[6.22e-10 6.09e-10 4.54e-09]]
fake:  [[3.24e-02 2.65e-09 6.35e-05]]
fake:  [[1.36e-01 1.12e-09 5.37e-09]]
fake:  [[2.16e-08 9.89e-01 5.51e-07]]
fake:  [[0.15 0.   0.08]]
fake:  [[0.99 0.57 0.8 ]]
fake:  [[1.11e-08 7.26e-09 6.62e-09]]
fake:  [[3.13e-09 4.52e-09 2.43e-09]]
fake:  [[2.36e-09 2.17e-09 5.42e-10]]
fake:  [[6.31e-10 2.93e-09 4.29e-09]]
[1,   320] segmentation loss: 13.942
fake:  [[6.50e-10 2.30e-09 4.98e-10]]
fake:  [[6.58e-10 2.01e-09 2.90e-09]]
fake:  [[2.88e-09 1.39e-09 6.70e-10]]
fake:  [[6.67e-10 3.84e-09 2.85e-09]]
fake:  [[2.43e-09 2.87e-09 4.79e-10]]
fake:  [[3.00e-09 6.26e-10 6.26e-10]]
fake:  [[5.93e-10 7.77e-10 1.74e-09]]
fake:  [[6.19e-10 2.28e-09 2.74e-09]]
fake:  [[2.43e-09 5.60e-10 7.12e-10]]
fake:  [[4.30e-09 2.10e-09 2.16e-09]]


valid:  [[1. 1. 1.]] ; fake:  [[1.13e-13 1.53e-11 3.08e-13]]
valid:  [[1. 1. 1.]] ; fake:  [[2.48e-12 2.10e-13 9.18e-12]]
[1,   470] discrimination loss: 0.001
valid:  [[1. 1. 1.]] ; fake:  [[1.47e-12 6.89e-13 2.39e-11]]
valid:  [[1. 1. 1.]] ; fake:  [[4.62e-12 2.29e-12 1.37e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[1.94e-13 5.09e-12 2.35e-13]]
valid:  [[1. 1. 1.]] ; fake:  [[4.06e-12 2.94e-12 8.49e-13]]
valid:  [[1. 1. 1.]] ; fake:  [[1.10e-12 2.67e-11 2.63e-13]]
valid:  [[1. 1. 1.]] ; fake:  [[4.48e-13 1.72e-12 3.90e-13]]
valid:  [[1. 1. 1.]] ; fake:  [[2.72e-12 4.41e-12 3.17e-13]]
valid:  [[1. 1. 1.]] ; fake:  [[5.55e-12 5.27e-13 4.05e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[4.29e-12 3.69e-13 3.97e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[8.37e-13 3.04e-11 2.29e-11]]
[1,   480] discrimination loss: 0.000
valid:  [[1. 1. 1.]] ; fake:  [[8.57e-12 1.31e-11 1.06e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[5.78e-12 7.52e-13 3.04e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[3.64e-11 2.70e-12 7.18e-12]]
valid:  [

valid:  [[1. 1. 1.]] ; fake:  [[3.25e-13 1.88e-05 4.21e-13]]
[1,   630] discrimination loss: 0.000
valid:  [[1. 1. 1.]] ; fake:  [[5.06e-12 6.80e-12 3.51e-12]]
valid:  [[1.   0.98 1.  ]] ; fake:  [[3.87e-03 8.68e-12 3.82e-13]]
valid:  [[1. 1. 1.]] ; fake:  [[3.91e-13 8.76e-06 4.42e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[7.08e-09 2.20e-10 1.20e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[1.05e-09 7.34e-08 5.00e-08]]
valid:  [[1. 1. 1.]] ; fake:  [[1.80e-11 4.28e-07 7.16e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[3.66e-12 2.08e-12 1.09e-10]]
valid:  [[1. 1. 1.]] ; fake:  [[3.79e-07 2.30e-05 5.51e-10]]
valid:  [[1. 1. 1.]] ; fake:  [[4.73e-03 6.17e-13 7.67e-06]]
valid:  [[1. 1. 1.]] ; fake:  [[3.76e-06 3.71e-12 4.46e-05]]
[1,   640] discrimination loss: 0.001
valid:  [[1. 1. 1.]] ; fake:  [[1.78e-12 1.59e-04 7.98e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[2.40e-12 2.91e-07 1.13e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[1.83e-07 4.60e-11 4.79e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[6.31e-12 4.00e-07 5.76e-07]]
val

...

valid:  [[1.   0.93 0.99]] ; fake:  [[1.99e-02 5.11e-09 8.07e-06]]
valid:  [[0.86 0.99 0.98]] ; fake:  [[3.71e-10 7.56e-06 3.34e-04]]
valid:  [[1.   0.98 0.94]] ; fake:  [[4.09e-11 8.52e-04 1.57e-08]]
valid:  [[0.99 1.   0.96]] ; fake:  [[5.56e-04 3.86e-08 7.20e-05]]
valid:  [[0.99 0.9  1.  ]] ; fake:  [[2.86e-12 2.60e-04 9.74e-10]]
valid:  [[1.   0.98 1.  ]] ; fake:  [[2.94e-02 3.69e-08 3.81e-04]]
valid:  [[0.97 0.98 1.  ]] ; fake:  [[2.20e-06 8.86e-05 6.41e-02]]
valid:  [[1.   0.98 0.99]] ; fake:  [[6.38e-01 8.98e-04 5.18e-04]]
[20, 17270] discrimination loss: 0.063
valid:  [[0.97 1.   0.99]] ; fake:  [[7.98e-07 4.84e-03 9.48e-08]]
valid:  [[0.99 0.99 0.98]] ; fake:  [[8.82e-03 1.50e-12 7.31e-04]]
valid:  [[1.   0.99 0.99]] ; fake:  [[2.78e-05 5.54e-04 5.03e-08]]
valid:  [[0.99 0.99 0.96]] ; fake:  [[2.37e-05 3.18e-01 3.80e-05]]
valid:  [[0.99 0.99 0.99]] ; fake:  [[0.01 0.   0.  ]]
valid:  [[0.98 0.97 0.86]] ; fake:  [[2.30e-03 2.61e-11 2.48e-12]]
valid:  [[0.96 0.96 0.99]] ; fake: 

valid:  [[1.   0.99 0.98]] ; fake:  [[9.06e-01 5.72e-03 3.88e-04]]
valid:  [[0.95 0.97 1.  ]] ; fake:  [[1.33e-05 1.92e-06 1.77e-06]]
valid:  [[0.98 1.   1.  ]] ; fake:  [[2.68e-10 9.74e-01 1.17e-04]]
valid:  [[0.98 1.   0.98]] ; fake:  [[2.11e-06 2.92e-02 3.00e-05]]
valid:  [[1.   0.97 0.99]] ; fake:  [[8.29e-04 9.10e-05 2.28e-08]]
valid:  [[0.91 0.93 0.99]] ; fake:  [[4.63e-03 7.49e-09 4.41e-06]]
valid:  [[0.99 0.87 0.98]] ; fake:  [[7.72e-04 7.73e-06 1.23e-05]]
valid:  [[0.99 1.   0.93]] ; fake:  [[1.92e-05 1.18e-02 3.01e-12]]
valid:  [[0.99 1.   0.98]] ; fake:  [[3.23e-07 9.45e-02 5.53e-06]]
valid:  [[0.87 0.98 0.96]] ; fake:  [[4.65e-10 2.19e-02 6.06e-06]]
[20, 17430] discrimination loss: 0.235
valid:  [[0.98 0.98 0.99]] ; fake:  [[8.28e-05 3.34e-05 9.53e-03]]
valid:  [[0.97 0.99 1.  ]] ; fake:  [[6.20e-06 2.17e-05 6.15e-01]]
valid:  [[1.   1.   0.33]] ; fake:  [[1.64e-02 2.21e-06 2.54e-07]]
valid:  [[1.   1.   0.91]] ; fake:  [[3.32e-07 2.14e-04 1.52e-06]]
valid:  [[0.98 0.99 0.8

fake:  [[6.95e-06 3.46e-06 3.66e-05]]
fake:  [[3.62e-04 3.03e-07 8.64e-02]]
fake:  [[9.71e-04 7.16e-02 2.70e-10]]
fake:  [[1.05e-05 2.45e-11 1.34e-04]]
fake:  [[6.21e-02 1.90e-09 7.69e-03]]
fake:  [[7.83e-06 1.15e-06 3.67e-02]]
fake:  [[3.25e-02 5.66e-04 4.34e-09]]
fake:  [[5.84e-09 6.13e-01 1.10e-01]]
fake:  [[6.84e-04 4.95e-05 8.15e-13]]
[20, 17570] segmentation loss: 10.806
fake:  [[5.39e-03 6.02e-07 2.03e-05]]
fake:  [[2.05e-01 2.21e-04 1.72e-05]]
fake:  [[6.96e-01 1.42e-08 1.33e-02]]
fake:  [[1.40e-09 1.93e-03 4.16e-06]]
fake:  [[2.67e-05 1.31e-08 2.28e-09]]
fake:  [[4.23e-04 2.64e-11 8.29e-05]]
fake:  [[1.71e-04 5.96e-07 4.53e-03]]
fake:  [[4.17e-05 1.02e-05 6.54e-06]]
fake:  [[4.41e-10 2.52e-04 1.61e-05]]
fake:  [[1.33e-12 2.08e-04 7.90e-01]]
[20, 17580] segmentation loss: 11.423
fake:  [[4.94e-04 2.16e-05 3.39e-07]]
fake:  [[5.89e-05 2.87e-06 2.10e-02]]
fake:  [[1.11e-04 6.16e-05 1.16e-04]]
fake:  [[6.84e-10 9.65e-02 9.84e-06]]
fake:  [[1.03e-08 8.01e-04 2.83e-05]]
fake:  [[3.1

valid:  [[1.   0.94 0.97]] ; fake:  [[1.81e-03 1.63e-07 3.66e-05]]
valid:  [[0.9  0.87 0.99]] ; fake:  [[3.61e-04 6.03e-07 8.10e-03]]
valid:  [[1.   0.98 1.  ]] ; fake:  [[1.02e-12 7.47e-04 6.97e-01]]
valid:  [[0.99 0.8  1.  ]] ; fake:  [[1.74e-03 5.85e-07 3.42e-03]]
valid:  [[0.98 1.   0.99]] ; fake:  [[1.53e-07 7.49e-02 4.58e-06]]
[20, 17700] discrimination loss: 0.076
[20, 17700] validation loss(seg): 0.063; validation loss(disc): 0.067
fake:  [[2.74e-08 2.18e-01 1.22e-02]]
fake:  [[1.88e-04 1.45e-07 9.82e-06]]
fake:  [[1.19e-04 4.33e-06 5.63e-05]]
fake:  [[3.16e-03 1.60e-04 3.26e-05]]
fake:  [[1.17e-03 9.99e-09 3.08e-08]]
fake:  [[9.42e-06 3.45e-08 2.30e-04]]
fake:  [[1.10e-02 9.62e-04 6.45e-10]]
fake:  [[6.41e-04 4.83e-05 3.88e-10]]
fake:  [[2.78e-01 3.88e-04 1.23e-05]]
fake:  [[2.28e-03 4.41e-08 2.38e-13]]
[20, 17710] segmentation loss: 11.319
fake:  [[3.52e-05 1.43e-03 1.50e-12]]
fake:  [[1.28e-06 8.10e-04 6.69e-01]]
fake:  [[2.12e-07 1.02e-12 5.33e-02]]
fake:  [[2.10e-06 1.38e-

valid:  [[0.98 0.94 0.95]] ; fake:  [[2.47e-08 4.18e-13 2.40e-04]]
valid:  [[0.99 1.   1.  ]] ; fake:  [[0.01 0.08 0.72]]
valid:  [[0.98 0.86 1.  ]] ; fake:  [[1.67e-07 9.39e-07 2.43e-05]]
valid:  [[1.   0.97 0.98]] ; fake:  [[2.07e-03 8.33e-05 5.41e-11]]
valid:  [[1. 1. 1.]] ; fake:  [[3.84e-13 1.35e-03 1.17e-01]]
valid:  [[0.99 0.99 1.  ]] ; fake:  [[2.88e-05 2.47e-03 1.29e-04]]
valid:  [[0.99 0.96 0.97]] ; fake:  [[2.45e-03 6.00e-10 1.13e-06]]
[20, 17860] discrimination loss: 0.077
valid:  [[1.   1.   0.89]] ; fake:  [[5.85e-13 2.13e-05 3.98e-05]]
valid:  [[0.99 0.95 0.97]] ; fake:  [[2.07e-12 3.61e-06 1.00e-01]]
valid:  [[1.   0.94 0.99]] ; fake:  [[3.42e-02 1.69e-10 5.91e-05]]
valid:  [[0.99 0.98 0.97]] ; fake:  [[1.68e-06 3.66e-04 4.59e-07]]
valid:  [[0.98 1.   0.99]] ; fake:  [[1.15e-07 3.22e-03 4.25e-07]]
valid:  [[1.   0.99 0.99]] ; fake:  [[2.76e-02 2.37e-05 1.41e-02]]
valid:  [[0.9  1.   0.93]] ; fake:  [[0. 0. 0.]]
valid:  [[1.   0.98 0.99]] ; fake:  [[1.92e-01 6.26e-06 6.9

In [6]:
#for python
torch.save(net.state_dict(), './unet_gan_usseg.pth')
torch.save(discriminator.state_dict(), './disc_gan_usseg.pth')
#for c++
traced_script_module = torch.jit.trace(net, img)
traced_script_module.save("./unet_gan_usseg_traced.pt")

## Inference

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

PATH = os.path.expanduser("~/workspace/us_robot/unet_gan_usseg.pth")
net = UNet(init_features=32).to(device)
net.load_state_dict(torch.load(PATH))
net = net.eval()


In [4]:
test_dir = os.path.expanduser("~/workspace/us_robot/DataSet/realDataSet/linear/vessel")
pred_dir = os.path.expanduser("~/workspace/us_robot/DataSet/realDataSet/linear/vessel_pred")

testset_list = os.listdir(test_dir)
testset_list = list(filter(lambda x: x.endswith('jpg'), testset_list))
resize_to=[256,256]
transform_image = transforms.Compose([
    transforms.Resize(resize_to),
    transforms.ToTensor(),
    transforms.Normalize(0.5,0.5) #Division by 255 is done, when the transformation assumes an image.
    ])
transform_label = transforms.Compose([
    transforms.Resize(resize_to),
    transforms.ToTensor()
    ])
invtransform_label = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([512,512])
    ])

for sample in testset_list:
    image_path = os.path.join(test_dir,sample)
    label_path = os.path.join(pred_dir,sample)
    
    img = Image.open(image_path).convert("L")
    img = transform_image(img)
    img = img.to(device)
    img = img.unsqueeze(0)
    #label = Image.open(label_path)
    #label = transform_label(label).to(device)
    #label = label.unsqueeze(0)
    
    
    with torch.no_grad():
        pred = net(img)
    
    #DiceIndex = (1 - DiceLoss(pred,label)).cpu().item()

    pred = invtransform_label(pred.cpu().squeeze(0))
    #fname = "pred%.2f.png"%DiceIndex
    fname = sample
    sav_path = os.path.join(pred_dir,fname)
    pred.save(sav_path)

In [5]:
test_dir = os.path.expanduser("~/workspace/us_robot/DataSet/SimRealDatasetTest")
testset_list = os.listdir(test_dir)
resize_to=[256,256]
transform_image = transforms.Compose([
    transforms.Resize(resize_to),
    transforms.ToTensor(),
    transforms.Normalize(0.5,0.5) #Division by 255 is done, when the transformation assumes an image.
    ])
transform_label = transforms.Compose([
    transforms.Resize(resize_to),
    transforms.ToTensor()
    ])
invtransform_label = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([512,512])
    ])
for sample in testset_list:
    image_path = os.path.join(test_dir,sample,"image.png")
    label_path = os.path.join(test_dir,sample,"label.png")
    
    img = Image.open(image_path)
    img = transform_image(img)
    img = img.to(device)
    img = img.unsqueeze(0)
    label = Image.open(label_path)
    label = transform_label(label).to(device)
    label = label.unsqueeze(0)
    
    
    with torch.no_grad():
        pred = net(img)
    
    DiceIndex = (1 - DiceLoss(pred,label)).cpu().item()

    pred = invtransform_label(pred.cpu().squeeze(0))
    fname = "pred%.2f.png"%DiceIndex
    sav_path = os.path.join(test_dir,sample,fname)
    pred.save(sav_path)

In [4]:
BCELoss = nn.BCELoss()

In [7]:
BCELoss(torch.Tensor([1,1,1]),torch.Tensor([0.1,0.1,0.1]))

tensor(90.)

TypeError: forward() missing 1 required positional argument: 'mask'

In [ ]:
print(summary(discriminator, {'input':torch.zeros((1, 1, 256, 256)),'mask':torch.zeros((1, 1, 256, 256))}, show_input=False))